In [41]:
using Downloads, CSV, DataFrames
parent_dir = "C:\\Users\\engin\\Documents\\GitHub\\Energy\\"
total_cross_section_dir = parent_dir * "ImportedData\\tendl_total_neutron_sigma_data\\"
neutron_production_dir = parent_dir * "ImportedData\\my_tendl_neutron_production_data\\"
neutron_MTs_file = parent_dir * "ExportedData\\produce_only_neutrons_MTs.csv"

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\ExportedData\\produce_only_neutrons_MTs.csv"

In [6]:
#Import a dictionary of MTs to know what data to download
MT_df = CSV.read(neutron_MTs_file, DataFrame, stringtype=String)
MT_dict = Dict([])
rows, cols = size(MT_df)
for row in 1:rows
  MT_df[row, 2] = String(MT_df[row, 2][2:end-1])
  MTs = [parse(Int64, n) for n in split(MT_df[row, 2], ",")]
  for MT in MTs
      MT_dict[MT] = MT_df[row, 1] 
  end
end
MT_dict

Dict{Any, Any} with 10 entries:
  152 => "(0, -4)"
  875 => "(0, -1)"
  50  => "(0, -1)"
  16  => "(0, -1)"
  891 => "(0, -1)"
  876 => "(0, -1)"
  38  => "(0, -2)"
  21  => "(0, -1)"
  17  => "(0, -2)"
  37  => "(0, -3)"

In [44]:
nuclide_df_path = parent_dir * "ExportedData\\nuclide_list.csv"
nuclide_df = CSV.read(nuclide_df_path, DataFrame, stringtype=String)
function get_mass_name(nuclide)
    index = 1
    while (tryparse(Int64, string(nuclide[index])) != nothing && index <= length(nuclide))
        index += 1
    end
    mass = tryparse(Int64, nuclide[1:index-1])
    name =  nuclide[index:end]
    name_1 = uppercase(name[1])
    if (length(name) > 1)
        name = name_1 * name[2]
    else 
      name = name_1
    end
    return mass, name
end
nuclide_list = [get_mass_name(nuclide) for nuclide in nuclide_df[!,1]]

5842-element Vector{Tuple{Int64, Any}}:
 (1, 'H')
 (2, 'H')
 (3, 'H')
 (3, "He")
 (3, "Li")
 (4, 'H')
 (4, "He")
 (4, "Li")
 (5, 'H')
 (5, "He")
 (5, "Li")
 (5, "Be")
 (6, 'H')
 ⋮
 (291, "Lv")
 (291, "Ts")
 (292, "Mc")
 (292, "Lv")
 (292, "Ts")
 (293, "Lv")
 (293, "Lv")
 (293, "Ts")
 (293, "Og")
 (294, "Ts")
 (294, "Og")
 (295, "Og")

In [15]:
function get_file_path(parent_dir, mt, mass, elem)
    mt_str, mass_str = lpad(string(mt), 3, "0"), lpad(string(mass), 3, "0")
    file_path = parent_dir * "n-" * elem * mass_str * "-" * "MT" * mt_str * ".csv"
    return file_path
end
function get_file_path(parent_dir, mt, nuclide)
    mass, elem = get_mass_name(nuclide)
    mt_str, mass_str = lpad(string(mt), 3, "0"), lpad(string(mass), 3, "0")
    file_path = parent_dir * "n-" * elem * mass_str * "-" * "MT" * mt_str * ".csv"
    return file_path
end
get_file_path(total_cross_section_dir, 1, nuclide_df[1,1])

"C:\\Users\\engin\\Documents\\GitHub\\Energy\\ImportedData\\tendl_total_neutron_sigma_data\\n-H001-MT001.csv"

In [42]:
function get_total_neutron_cross_section(nuclide)
    file_path = get_file_path(total_cross_section_dir, 1, nuclide)
    return CSV.read(file_path, DataFrame)
end
function get_total_neutron_cross_section(mass, elem)
    file_path = get_file_path(total_cross_section_dir, 1, mass, elem)
    return CSV.read(file_path, DataFrame)
end
get_total_neutron_cross_section(nuclide_list[1][1], nuclide_list[1][2])

,E(MeV),xs(mb)
,Float64,Float64
1,1.0e-11,1.1778e6
2,1.03223e-11,1.15927e6
3,1.06549e-11,1.14104e6
4,1.09984e-11,1.12309e6
5,1.13527e-11,1.10542e6
6,1.17186e-11,1.08803e6
7,1.20962e-11,1.07092e6
8,1.24861e-11,1.05408e6
9,1.28884e-11,1.0375e6


In [43]:
function get_neutron_production_cross_sections(nuclide)
    mass, elem = get_mass_name(nuclide)
    mass_str = lpad(string(mass), 3, "0")
    file_paths = [neutron_production_dir * file_name 
                for file_name in neutron_production_file_names 
                if occursin(elem * mass_str, file_name)]
    return [CSV.read(file_path, DataFrame) for file_path in file_paths]
end
function get_neutron_production_cross_sections(mass, elem)
    mass_str = lpad(string(mass), 3, "0")
    file_paths = [neutron_production_dir * file_name 
                for file_name in neutron_production_file_names 
                if occursin(elem * mass_str, file_name)]
    return [CSV.read(file_path, DataFrame) for file_path in file_paths]
end
get_neutron_production_cross_sections(nuclide_list[2][1], nuclide_list[2][2])

1-element Vector{DataFrame}:
 124×2 DataFrame
 Row │ E(MeV)   xs(mb)   
     │ Float64  Float64  
─────┼───────────────────
   1 │   3.339   0.0
   2 │   3.4     1.2
   3 │   3.5     2.9
   4 │   3.75    7.10318
   5 │   4.0    13.5
   6 │   4.25   19.0312
   7 │   4.5    25.0
   8 │   5.0    37.0
   9 │   5.5    48.6414
  10 │   5.6    51.0236
  11 │   6.0    60.2
  ⋮  │    ⋮        ⋮
 115 │ 132.0    69.4702
 116 │ 134.0    68.8945
 117 │ 136.0    68.3321
 118 │ 138.0    67.7825
 119 │ 140.0    67.2451
 120 │ 142.0    66.7194
 121 │ 144.0    66.2049
 122 │ 146.0    65.7011
 123 │ 148.0    65.2076
 124 │ 150.0    64.724
         103 rows omitted

In [82]:
file_path = neutron_production_dir * neutron_production_file_names[2]
function neutrons_produced(file_path)
    MT = tryparse(Int64, split(split(file_path, "MT")[end], ".")[1])
    neutrons = tryparse(Int64, split(MT_dict[MT], ",")[end][1:end-1]) * -1
    return neutrons
end
neutrons_produced(file_path)

2